<script type="text/x-mathjax-config">
MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
MathJax.Ajax.config.path["mhchem"] = "https://cdnjs.cloudflare.com/ajax/libs/mathjax-mhchem/3.3.2";
MathJax.Hub.Config({TeX: {extensions: ["[mhchem]/mhchem.js"]}});
</script><script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML' async></script>
$$\require{mhchem}$$

## Using a Jupyter notebook

This is a Jupyter notebook, that is, a collection of formatted text and live Python code.

Text and code are separated into cells. The active cell is highlighted with a border with a thicker left edge in blue or green. You can activate a different cell by clicking it.

To run all of the code within the active cell, just press Ctrl and Enter. `In [*]:` will appear at the top left corner while the code is running, and the `*` will turn into a number once it has finished. Any output from the code will then appear beneath the cell. *If you find at any point one of the text cells (e.g. this one) is no longer nicely formatted, but instead looks like code, just do the same: select it and press Ctrl and Enter to revert back.*

In the code, lines beginning with `#` are just comments. We use comment lines with arrows to indicate which parts of the code you are encouraged to modify:

```python
# Don't change this bit!
#-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓

# Do edit this section!

#-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑
# Don't edit this part either!
```

Code cells need to be run in the order that they appear in the notebook, as the later cells depend on the results of calculations from earlier cells.

---

# Total pH in a tris buffer solution

The examples below are computed by Pytzer, which is a Python implementation of the Pitzer model for chemical activities in aqueous solutions. Although it's still in development, there is [full documentation](https://pytzer.readthedocs.io/en/latest/) including [detailed installation instructions](https://pytzer.readthedocs.io/en/latest/quick-start/), and [the code is freely available](https://github.com/mvdh7/pytzer) for you to use.

## Define the solution composition and conditions

The first thing we need to do is to define the conditions (i.e. temperature and pressure) that the solution is under, and the molality of each solute dissolved within it (i.e. its composition). The molalities are divided into fixed values for pH-conservative solutes, and 'total' values for species that will be allowed to equilibrate.

The included equilibria are:

$$\ce{trisH+} \rightleftharpoons \ce{tris} + \ce{H+}$$

$$\ce{HSO4-} \rightleftharpoons \ce{SO4^2-} + \ce{H+}$$

$$\ce{Mg^2+} + \ce{OH-} \rightleftharpoons \ce{MgOH+}$$

$$\ce{H2O} \rightleftharpoons \ce{OH-} + \ce{H+}$$

In other words, this represents the Pitzer model formulated by Waters and Millero (2013), plus tris buffer.

> Waters, J. F., and Millero, F. J. (2013). The free proton concentration scale for seawater pH. *Mar. Chem.* 149, 8–22. [doi:10.1016/j.marchem.2012.11.003](https://doi.org/10.1016/j.marchem.2012.11.003).

In the first cell of code below, we define the molalities (in mol/kg-H<sub>2</sub>O) of each solute in the `solutes` [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries). Fixed-concentration ions are specified simply using the relevant chemical symbol (e.g. `'Na'` for $\ce{Na+}$), whereas total concentrations for equilibrating species are prefaced with `t_`, so:

`solutes['t_HSO4']` $= [\ce{HSO4-}] + [\ce{SO4^2-}]$

`solutes['t_trisH']` $= [\ce{trisH+}] + [\ce{tris}]$

`solutes['t_Mg']` $= [\ce{Mg^2+}] + [\ce{MgOH+}]$

There is no need to specify $[\ce{H+}]$ or $[\ce{OH-}]$, as these are determined by the solver itself.

Select the code cell below and run it by pressing Ctrl and Enter together. The concentrations that have been defined should print out beneath the cell.

In [ ]:
from numpy import log10, sqrt
import pytzer as pz
import pytzertools as pzt
solutes = {}
#-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓

# Set the temperature (in K) and pressure (in dbar)
tempK = 298.15
pres = 10.10325

# Define the solutes and their molalities in mol/kg-H2O
# First, the fixed-molality ions:
solutes['Na'] = 0.44516
solutes['Ca'] = 0.01077
solutes['K' ] = 0.01058
solutes['Cl'] = 0.56912
# Then, solutes involved in equilibria:
solutes['t_HSO4'] = 0.02926
solutes['t_trisH'] = 0.08
solutes['t_Mg'] = 0.05518

# Add some extra sodium sulfate, for example
extra_Na2SO4 = 0.0
solutes['Na'] += extra_Na2SO4*2
solutes['t_HSO4'] += extra_Na2SO4

#-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑
# Convert everything into arrays for Pytzer, and print out the concentrations to check they look sensible
solutearrays = pzt.solutes2arrays(solutes)
pzt.printmols(*solutearrays)

You may note that after the solute concentrations are originally defined, there are some extra lines of code to 'add some extra sodium sulfate'. The molality of extra sodium sulfate to add is set by the variable `extra_Na2SO4`, and you can see that this is added to the relevant entries in `solutes` in the subsequent two lines, taking into account the stoichiometry of the electrolyte.

By default, the amount of extra sodium sulfate added is set to zero. You could try changing this value and seeing the effect on the calculations in the later code cells. You could also try adding a set amount of a different electrolyte by replicating and adjusting these three lines of code appropriately. For example, to add 0.1 mol/kg-H<sub>2</sub>O of KCl, you could add the following to the above code cell:

```python
extra_KCl = 0.1
solutes['K'] += extra_KCl
solutes['Cl'] += extra_KCl
```

## Solve for equilibrium

Once the solution's composition and conditions have been defined and the code cell above executed, we can solve for the solution's equilibrium speciation. Running the following cell will do just that, and will then print out the equilibrium composition beneath:

In [ ]:
# Calculate and display equilibrium speciation
allmols, allions = pzt.solve(solutearrays, tempK, pres)
pzt.printmols(allions, allmols)

## Calculate Total scale pH

Our aim is to calculate the pH on the Total scale, defined as:

$$\text{pH}_\text{T} = -\log_{10}([\ce{H+}] + [\ce{HSO4-}])$$

Now that we have calculated the equilibrium speciation in the cell above, we can just pull out the hydrogen and bisulfate ion molalities from the equilibrium calculation results and put them into the above equation:

In [ ]:
# Extract H+ and HSO4- molalities from equilibrium results
mH = allmols[allions == 'H']
mHSO4 = allmols[allions == 'HSO4']

# Add together & take the logarithm to get Total scale pH
pH_Total = -log10(mH + mHSO4)
print('Total scale pH = {:.3f}'.format(pH_Total[0]))

At this point, you could go back to the first code cell and modify the solution composition, for example by increasing the amount of extra $\ce{Na2SO4}$ that is added. If you then run the first three cells again in order, you will see the effect of the changing composition on the Total scale pH calculated here.

## Uncertainty propagation

The next step is to propagate uncertainties in the thermodynamic equilibrium constants for the equilibria in this system through to Total scale pH to see their relative importance. Uncertainty propagation requires knowing the derivative of the target variable with respect to each uncertain input. *Computationally, this step is a little slow as it involves calculating finite-difference gradients over a least-squares solver.*

In [ ]:
# Calculate derivative of pH w.r.t. each ln(K)
pHgrads = pzt.pHgrads(solutearrays, tempK, pres)
print('Finished!')

Once we have those derivatives, we simply multiply them by the estimated uncertainty in each thermodynamic equilibrium constant to quantify the corresponding uncertainty in pH:

In [ ]:
uncert_lnk = {}
#-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓-↓

# Define uncertainties
uncert_lnk['HSO4'] = 0.0484
uncert_lnk['trisH'] = 0.02 # GUESSED PLACEHOLDER VALUE!!!
uncert_lnk['MgOH'] = 0.022
uncert_lnk['H2O'] = 0.023

#-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑-↑
# Propagate and print out uncertainties
uncert_pHT = {eq: abs(uncert_lnk[eq]*pHgrads[eq]) for eq in pHgrads}
total_uncert_pHT = sqrt(sum([uncert_pHT[eq]**2 for eq in uncert_pHT]))
for eq in pHgrads:
    print('±{:.6f} = uncertainty in Total pH due to K({})'.format(uncert_pHT[eq], eq))
print('\n±{:6f} = total uncertainty in Total pH due to all thermodynamic Ks'.format(total_uncert_pHT))